## Imports

In [3]:
import torch
from torch import nn
from torch.nn import functional as F

import dlc_practical_prologue

## Load data

In [4]:
N = 1000

x_train, y_train, y_train_classes, x_test, y_test, y_test_classes = \
dlc_practical_prologue.generate_pair_sets(N)

assert x_train.shape == torch.Size([N, 2, 14, 14])
assert y_train.shape == torch.Size([N])
assert y_train_classes.shape == torch.Size([N, 2])
assert x_test.shape == torch.Size([N, 2, 14, 14])
assert y_test.shape == torch.Size([N])
assert y_test_classes.shape == torch.Size([N, 2])

## Define model

In [79]:
class model_1(nn.Module):
    def __init__(self):
        super(model_1, self).__init__()
        self.conv1 = nn.Conv2d(2, 10, kernel_size=3)
#         self.maxpool1 = nn.MaxPool2d(2)
        
        self.conv2 = nn.Conv2d(10, 10, kernel_size=3)
#         self.maxpool2 = nn.MaxPool2d(2)

        self.dense1 = nn.Linear(40, 10)
        self.dense2 = nn.Linear(10, 1)
        
    def forward(self, x):
#         x = self.maxpool1(self.conv1(x))
#         x = self.maxpool2(self.conv2(x))

        x = F.max_pool2d(self.conv1(x), kernel_size=2)
        x = F.max_pool2d(self.conv2(x), kernel_size=2)
        x = F.relu(self.dense1(x.view(-1)))
        x = F.sigmoid(self.dense2(x))
        return x

## Training/Evaluation function

In [80]:
def train_model(model, train_input, train_target, epochs):
    # Inspired by exercise corrige 
    model.train()
    
    criterion = nn.BCELoss()
    eta = 0.01

    for e in range(epochs):
        sum_loss = 0
        for i in range(0, train_input.size(0)):
            output = model(train_input[i:i+1])[0]
            loss = criterion(output, train_target[i:i+1].float())
            model.zero_grad()
            loss.backward()
            sum_loss = sum_loss + loss.item()

            for p in model.parameters():
                p -= eta * p.grad

In [81]:
def evaluate_model(model, test_input, test_target):
    model.eval()
    criterion = nn.BCELoss()
    preds_proba = model(test_input)
    print(preds_proba.shape)
    
    loss = criterion(preds_proba, test_target.float()).item()
    
    accuracy = (preds == test_target).sum().item()/preds.size(0)
    #accuracy = sum([pred == truth for pred, truth in zip(preds, test_target)])
    return loss, accuracy

In [82]:
def mean(x):
    return sum(x)/len(x)

def var(x):
    u = mean(x)
    return sum([(loss-u)**2 for loss in x])/len(x)

## Experiment

In [86]:
for p in model.parameters():
    print(p.shape)

torch.Size([10, 2, 3, 3])
torch.Size([10])
torch.Size([10, 10, 3, 3])
torch.Size([10])
torch.Size([10, 40])
torch.Size([10])
torch.Size([1, 10])
torch.Size([1])


In [83]:
number_training = 10
epochs = 25

losses = []
accuracies = []
for i_train in range(number_training):
    
    model = model_1()
    indices_shuffle = torch.randperm(N)
    
    train_model(model, 
                x_train[indices_shuffle],
                y_train[indices_shuffle],  
                epochs = epochs)
    
    loss, accuracy = evaluate_model(model, x_test, y_test)
    print("Attempt", i_train, ": loss", loss, "- accuracy", accuracy)
    losses.append(loss)
    accuracies.append(accuracy)

print("Experiment results :")
print("Loss mean : %.2f (%.2f)" % (mean(losses), var(losses)))
print("Accuracy mean : %.2f (%.2f)" % (mean(accuracies), var(accuracies)))

C:\Users\SimonRoquette\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
C:\Users\SimonRoquette\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\loss.py:516: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


RuntimeError: a leaf Variable that requires grad is being used in an in-place operation.